In [8]:
# Install the necessary packages if not already installed
#pip install s3fs
#pip install nltk
#pip install wordcloud
#pip install seaborn

# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load S3 Bucket into Pandas DataFrame

In [9]:
# Read in the data from S3 to dataframe
biden_df = pd.read_csv('s3://bidentweetscsv/hashtag_joebiden.csv')
biden_df.head()

C:\Users\deolg\AppData\Local\Temp\ipykernel_12676\2768635924.py:2: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  biden_df = pd.read_csv('s3://bidentweetscsv/hashtag_joebiden.csv')


,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.31652929585929e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,809904438.0,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.3165293050069524e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3494182277.0,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.3165293080815575e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596012018524e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529312741253e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032806955356545e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


# Clean DataFrame

In [10]:
biden_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059909 entries, 0 to 1059908
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   created_at            1059909 non-null  object 
 1   tweet_id              887173 non-null   object 
 2   tweet                 887173 non-null   object 
 3   likes                 887092 non-null   object 
 4   retweet_count         887073 non-null   float64
 5   source                886360 non-null   object 
 6   user_id               887067 non-null   object 
 7   user_name             887048 non-null   object 
 8   user_screen_name      786660 non-null   object 
 9   user_description      704647 non-null   object 
 10  user_join_date        776784 non-null   object 
 11  user_followers_count  776885 non-null   object 
 12  user_location         543063 non-null   object 
 13  lat                   355284 non-null   object 
 14  long                  355284 non-n

In [ ]:
missing_counts = biden_df.sum()
print(missing_counts)


In [ ]:
# Remove unnecessary columns
biden_df = biden_df.drop(columns=['tweet_id', 'source', 'user_id', 'user_name', 'user_screen_name', 'user_description', 'user_join_date', 'user_location', 'lat', 'long', 'city', 'continent', 'state_code', 'collected_at'])

# Display Dataframe
biden_df.head()

In [ ]:
# Drop rows that are not from the United States of America
biden_df = biden_df[biden_df['country'] == 'United States of America']

# Display Dataframe
biden_df.head()

In [ ]:
# Drop rows with NaN values
biden_df = biden_df.dropna()

# Display Dataframe
biden_df.head()

In [ ]:
# Get info on the dataframe
biden_df.info()

In [ ]:
# Convert 'likes' and 'user_followers_count' to numeric
biden_df['likes'] = pd.to_numeric(biden_df['likes'], errors='coerce')
biden_df['user_followers_count'] = pd.to_numeric(biden_df['user_followers_count'], errors='coerce')

# Convert 'created_at' to datetime
biden_df['created_at'] = pd.to_datetime(biden_df['created_at'], errors='coerce')

In [ ]:
# Confirm changes
biden_df.info()

# Clean Tweets

In [ ]:
# Text cleaning
import re

def clean_tweet_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\S+', '', text)     # Remove mentions
    text = re.sub(r'#\S+', '', text)     # Remove hashtags
    text = re.sub(r'[^a-z\s]', '', text) # Remove non-alphabetic characters
    return text

biden_df['cleaned_tweet'] = biden_df['tweet'].apply(clean_tweet_text)


In [ ]:
# Remove rows where 'tweet' is an empty string
biden_df = biden_df[biden_df['cleaned_tweet'].astype(str).str.strip() != '']

In [ ]:
# Display Dataframe
biden_df.head()

In [ ]:
# Import the NLTK library and download the VADER lexicon
import nltk
nltk.download('vader_lexicon')

In [ ]:
# Import the SentimentIntensityAnalyzer class from the NLTK library
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to DataFrame
biden_df['sentiment'] = biden_df['cleaned_tweet'].apply(get_vader_sentiment)

In [ ]:
# Display Dataframe
biden_df.head()

In [ ]:
# Save the cleaned data to a new CSV file
biden_df.to_csv('Resources/hashtag_joebiden_cleaned.csv', index=False)

# Correlation

In [ ]:
# Assign numerical values to sentiments (e.g., positive: 1, neutral: 0, negative: -1)
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
biden_df['sentiment_numeric'] = biden_df['sentiment'].map(sentiment_mapping)

In [ ]:
biden_df.head()

In [ ]:
# Selecting relevant columns for correlation
columns_for_correlation = ['likes', 'retweet_count', 'user_followers_count', 'sentiment_numeric']
correlation_matrix = biden_df[columns_for_correlation].corr()

# Display the correlation matrix
print(correlation_matrix)

In [ ]:
# Visualize the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Sentiment and Engagement Metrics')
plt.show()

# Regression Analysis

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Preparing data for regression analysis
X = biden_df[['likes', 'retweet_count', 'user_followers_count']]
y = biden_df['sentiment_numeric']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate metrics
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
R2 = r2_score(y_test, y_pred)

# Print the metrics
print(f"Mean Absolute Error: {MAE}")
print(f"Mean Squared Error: {MSE}")
print(f"R-squared Score: {R2}")

# Step 2: Logistic Regression Model


 understand how the textual content of tweets reflects sentiment

In [ ]:
#  'cleaned_tweet' is feature column and 'sentiment' is the target
X =biden_df['cleaned_tweet'] #Features (X)

y =biden_df['sentiment_numeric'] #Target (y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Create a pipeline with TF-IDF Vectorizer and Logistic Regression
model = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=1000))

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")


# Step 3: Moving to BERT with Hugging Face Transformers

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Example of tokenizing input for BERT
def encode_examples(texts, labels, tokenizer, max_length=512, pad_to_max_length=True):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for text in texts:
        bert_input = tokenizer.encode_plus(text,
                                           add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
                                           max_length=max_length,  # Max length to truncate/pad
                                           pad_to_max_length=pad_to_max_length,  # Pad sentence to max length
                                           return_attention_mask=True,  # Return attention mask
                                           )
        
        input_ids.append(bert_input['input_ids'])
        attention_masks.append(bert_input['attention_mask'])
        token_type_ids.append(bert_input['token_type_ids'])

    # Convert to tensors
    input_ids = tf.constant(input_ids)
    attention_masks = tf.constant(attention_masks)
    token_type_ids = tf.constant(token_type_ids)
    labels = tf.keras.utils.to_categorical(labels, num_classes=3)  # Assuming 3 classes for sentiment

    return input_ids, attention_masks, token_type_ids, labels

# Preparing dataset
# Assuming 'texts' is a list of tweets and 'labels' is a list of numerical sentiment labels
input_ids, attention_masks, token_type_ids, labels = encode_examples(biden_df['cleaned_tweet'].tolist(), 
                                                                     biden_df['sentiment_numeric'].tolist(),
                                                                     tokenizer)

# Define the dataset using TensorFlow
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, 
                                               "attention_mask": attention_masks, 
                                               "token_type_ids": token_type_ids}, 
                                              labels))

# Splitting the dataset, training the model, and evaluation would follow here


To address the ImportError you're encountering and proceed with using TensorFlow for your sentiment analysis task, you'll need to adapt your code to use TensorFlow's version of BERT for sequence classification, which is TFBertForSequenceClassification. This adjustment is necessary since your environment currently supports TensorFlow, not PyTorch. Below is an example of how to adjust your code to use TensorFlow classes for loading and utilizing a BERT model for sequence classification.

# pip install transformers


# Visualizations

In [ ]:
# Visualization of sentiment distribution
sns.countplot(x='sentiment', data=biden_df)
plt.title('Sentiment Distribution of Biden Tweets')
plt.show()

In [ ]:
# Scatter plot of actual vs predicted values
plt.scatter(X_test['likes'], y_test, color='blue', label='Actual', alpha=0.5)
plt.scatter(X_test['likes'], y_pred, color='red', label='Predicted', alpha=0.5)

# Line of best fit
m, b = np.polyfit(X_test['likes'], y_pred, 1)
plt.plot(X_test['likes'], m*X_test['likes'] + b, color='green')

plt.xlabel('Likes')
plt.ylabel('Sentiment Score')
plt.title('Actual vs Predicted Sentiment Scores')
plt.legend()
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all the tweets into a single string
text = ' '.join(biden_df['cleaned_tweet'])  # Assuming 'cleaned_tweet' is your column name
wordcloud = WordCloud(width = 800, height = 800, 
                      background_color ='white', 
                      min_font_size = 10).generate(text)

# Plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()